In [1]:
# first, REMEMBER to activate cryptoalgowheel-S2 environment!

In [2]:
import datetime
import os
import sys

import backtrader as bt
import backtrader.analyzers as btanalyzers
import numpy as np
import pandas as pd
import matplotlib
import PyQt5
import seaborn
import sklearn

In [3]:
#*****WARNING: REVISE THE "dir" FOLDER PATHS!!!
datadir = "./data"
logdir = "./log"
reportdir = "./report"
datafile = "BTC_USDT_1h.csv"      #!NOTICE: use our data "BTC_USDT_1h.csv" here
from_datetime = "2020-01-01 00:00:00"
to_datetime = "2020-04-01 00:00:00"

In [4]:
class OptDoubleSMACross(bt.Strategy):
    params = (
        ("pfast", 10),
        ("pslow", 20),
    )

    def log(self, txt, dt=None, doprint=False):     #(by default don't print log here)
        if doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print("%s, %s" % (dt.isoformat(), txt))

    def __init__(self):
        self.dataclose = self.datas[0].close

        # add both "fast" and "slow" SimpleMovingAverage indicators
        self.fastsma = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.pfast)
        self.slowsma = bt.indicators.SimpleMovingAverage(self.datas[0], period = self.params.pslow)
        # add a "CrossOver" signal!!
        self.crossover = bt.indicators.CrossOver(self.fastsma, self.slowsma) #NOTICE here passing in "fast" SMA as 1st line, "slow" SMA as 2nd line
        #["CrossOver" indicator Usage reference: https://www.backtrader.com/home/helloalgotrading/; documentation: https://www.backtrader.com/docu/indautoref/#crossover]
    
        # *****! Warning: avoid the erroreous cases of "pslow" larger than "pfast"!
        # *** Documentation: https://www.backtrader.com/docu/exceptions/#strategyskiperror
        if self.params.pslow < self.params.pfast+5:
            raise bt.errors.StrategySkipError

    def next(self):

        if not self.position:         #if not in the market yet (no "position" yet)
            if self.crossover > 0:     # "CrossOver" function return 1.0: meaning "fast SMA"(1st line) crosses the "slow SMA"(2nd line) upwards
                #--BUY!
                self.buy()
        else:           #("already in the market")
            if self.crossover < 0:     #"CrossOver" function return -1.0: meaning "fast SMA"(1st line) crosses the "slow SMA"(2nd line) downwards
                #--SELL!
                self.sell()

    #*** added "Strategy hook" here - "stop" method, in order to record the portfolio final net value of each optimization round:  
    def stop(self):
        self.log("Fast SMA Period %2d, Slow SMA Period %2d: Ending Value %.2f" %
        (self.params.pfast, self.params.pslow, self.broker.getvalue()), doprint=True)   #(do print the log message by the end of each optimization round here)
      

KPIs to be calculated: <br>
- Return (ending and starting values)
- MaxDrawDown
- TotalTrades (number of trades, WinTrades + LossTrades)
- WinTrades
- LossTrades
- WinRatio (WinTrades / TotalTrades)
- AverageWin$ (TotalWins dollar value / WinTrades)
- AverageLoss$ (TotalLosses dollar value / LossTrades)
- AverageWinLossRatio (AverageWin\$ / AverageLoss\$)

In [5]:
if __name__ == "__main__":
    cerebro = bt.Cerebro()

    # feed data:
    data = pd.read_csv(os.path.join(datadir, datafile), index_col="datetime", parse_dates=True)
    data = data.loc[(data.index >= pd.to_datetime(from_datetime)) & (data.index <= pd.to_datetime(to_datetime))]         #(just in case for the chosen time window here)
    datafeed = bt.feeds.PandasData(dataname=data, timeframe=bt.TimeFrame.Minutes, compression=60)                   #***! Notice detail: specify the time frame as "Hourly Data" as here
    cerebro.adddata(datafeed)

    # add an "optimization strategy"
    strats = cerebro.optstrategy(OptDoubleSMACross, pfast=range(5,21), pslow=range(10,51))  #optimizing grid here: "pfast" parameter from 5 to 20 & "pslow" parameter from ("pfast"+5) to 50
    #***IMPORTANT NOTICE of detail: "pslow" must be larger than "pfast"!!!(otherwise bugs)

    cerebro.addsizer(bt.sizers.PercentSizer, percents=99)

    cerebro.broker.setcash(10000)
    cerebro.broker.setcommission(commission=0.001)

    # [pending] ***** Add Analyzers!!!
    cerebro.addanalyzer(btanalyzers.TimeReturn, timeframe=bt.TimeFrame.NoTimeFrame, _name = "timereturn")
    cerebro.addanalyzer(btanalyzers.DrawDown, _name="drawdown")
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name="trade_comp")

    # add writer
    #cerebro.addwriter(bt.WriterFile, out="/Users/baixiao/Desktop/temp_opt.csv", csv=True)

    results = cerebro.run(maxcpus=1)          #[documentation: https://www.backtrader.com/docu/cerebro/#returning-the-results]

2020-04-01, Fast SMA Period  5, Slow SMA Period 10: Ending Value 6527.26
2020-04-01, Fast SMA Period  5, Slow SMA Period 11: Ending Value 6737.25
2020-04-01, Fast SMA Period  5, Slow SMA Period 12: Ending Value 7368.98
2020-04-01, Fast SMA Period  5, Slow SMA Period 13: Ending Value 7228.26
2020-04-01, Fast SMA Period  5, Slow SMA Period 14: Ending Value 6947.20
2020-04-01, Fast SMA Period  5, Slow SMA Period 15: Ending Value 7315.28
2020-04-01, Fast SMA Period  5, Slow SMA Period 16: Ending Value 7315.13
2020-04-01, Fast SMA Period  5, Slow SMA Period 17: Ending Value 7406.08
2020-04-01, Fast SMA Period  5, Slow SMA Period 18: Ending Value 8590.76
2020-04-01, Fast SMA Period  5, Slow SMA Period 19: Ending Value 8529.92
2020-04-01, Fast SMA Period  5, Slow SMA Period 20: Ending Value 8444.93
2020-04-01, Fast SMA Period  5, Slow SMA Period 21: Ending Value 10680.66
2020-04-01, Fast SMA Period  5, Slow SMA Period 22: Ending Value 10043.44
2020-04-01, Fast SMA Period  5, Slow SMA Period 2

optimization documentation: https://www.backtrader.com/docu/optimization-improvements/

In [6]:
### Extract optimization analyzer results ("KPIs")
result_df = []
for i in range(len(results)):
    try:
        name = "SMACross"
        sma_pfast = results[i][0].params.__dict__["pfast"]
        sma_pslow = results[i][0].params.__dict__["pslow"]
        rtn = list(results[i][0].analyzers.timereturn.get_analysis().values())[0]
        maxdrawdown = results[i][0].analyzers.drawdown.get_analysis().max.drawdown
        totaltrades = results[i][0].analyzers.trade_comp.get_analysis().total.closed
        wintrades = results[i][0].analyzers.trade_comp.get_analysis().won.total
        losstrades = results[i][0].analyzers.trade_comp.get_analysis().lost.total
        winratio = wintrades / totaltrades
        averagewinvalue = results[i][0].analyzers.trade_comp.get_analysis().won.pnl.average
        averagelossvalue = results[i][0].analyzers.trade_comp.get_analysis().lost.pnl.average
        averagewinlossratio = abs(averagewinvalue / averagelossvalue)
        longestwinstreak = results[i][0].analyzers.trade_comp.get_analysis().streak.won.longest
        longestlossstreak = results[i][0].analyzers.trade_comp.get_analysis().streak.lost.longest
        current = {"Name": name, "sma_pfast": sma_pfast, "sma_pslow": sma_pslow, "Return": round(rtn, 4), "MaxDrawDown": round(maxdrawdown, 4), "TotalTrades#": totaltrades, "WinTrades#": wintrades, "LossTrades#": losstrades, "WinRatio": round(winratio, 4), "AverageWin$": round(averagewinvalue, 4), "AverageLoss$": round(averagelossvalue, 4), "LongestWinStreak": longestwinstreak, "LongestLossStreak": longestlossstreak, "AverageWinLossRatio": round(averagewinlossratio, 4)}
        result_df.append(current)
    except:        #*****! Warning: there're certain indexes where the strategy has been "skipped" ("bt.errors.StrategySkipError" set previously) -- "list index out of range" error would happen in these places!!!
        pass

result_df = pd.DataFrame(result_df)

In [7]:
result_df

,Name,sma_pfast,sma_pslow,Return,MaxDrawDown,TotalTrades#,WinTrades#,LossTrades#,WinRatio,AverageWin$,AverageLoss$,LongestWinStreak,LongestLossStreak,AverageWinLossRatio
0,SMACross,5,10,-0.3473,43.6326,130,39,91,0.3000,179.6129,-115.1390,2,14,1.5600
1,SMACross,5,11,-0.3263,39.7296,126,39,87,0.3095,167.4799,-112.5800,3,7,1.4877
2,SMACross,5,12,-0.2631,35.8093,111,34,77,0.3063,192.6618,-119.2406,3,12,1.6157
3,SMACross,5,13,-0.2772,37.5877,102,34,68,0.3333,203.4041,-139.7284,3,11,1.4557
4,SMACross,5,14,-0.3053,39.2423,97,29,68,0.2990,232.6117,-142.0268,2,9,1.6378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,SMACross,20,46,-0.1041,32.2497,31,10,21,0.3226,408.9584,-239.0251,2,8,1.7109
532,SMACross,20,47,-0.0648,29.8744,32,8,24,0.2500,514.4823,-193.6284,2,7,2.6571
533,SMACross,20,48,-0.0557,30.1451,29,11,18,0.3793,369.3516,-250.1189,2,5,1.4767
534,SMACross,20,49,-0.0447,28.5999,29,10,19,0.3448,397.4869,-226.4542,2,5,1.7553


#### Compute the ranks for 4 KPIs and calculate final score (average of 4 ranks)

(Notice!: here use "minimum rank" to be assigned to **ties**)

rank of 4 KPIs:<br>
- rank of Return
- rank of MaxDrawDown
- rank of WinRatio
- rank of AverageWinLossRatio

In [10]:
result_df["RankReturn"]=result_df["Return"].rank(method="min", ascending=False).astype("int")
result_df["RankMaxDrawDown"]=result_df["MaxDrawDown"].rank(method="min", ascending=True).astype("int")
result_df["RankWinRatio"]=result_df["WinRatio"].rank(method="min", ascending=False).astype("int")
result_df["RankAverageWinLossRatio"]=result_df["AverageWinLossRatio"].rank(method="min", ascending=False).astype("int")

In [11]:
result_df["Score"]= (result_df["RankReturn"]+result_df["RankMaxDrawDown"]+result_df["RankWinRatio"]+result_df["RankAverageWinLossRatio"])/4

In [12]:
result_df

,Name,sma_pfast,sma_pslow,Return,MaxDrawDown,TotalTrades#,WinTrades#,LossTrades#,WinRatio,AverageWin$,AverageLoss$,LongestWinStreak,LongestLossStreak,AverageWinLossRatio,RankReturn,RankMaxDrawDown,RankWinRatio,RankAverageWinLossRatio,Score
0,SMACross,5,10,-0.3473,43.6326,130,39,91,0.3000,179.6129,-115.1390,2,14,1.5600,533,536,398,484,487.75
1,SMACross,5,11,-0.3263,39.7296,126,39,87,0.3095,167.4799,-112.5800,3,7,1.4877,531,527,353,495,476.50
2,SMACross,5,12,-0.2631,35.8093,111,34,77,0.3063,192.6618,-119.2406,3,12,1.6157,519,467,365,470,455.25
3,SMACross,5,13,-0.2772,37.5877,102,34,68,0.3333,203.4041,-139.7284,3,11,1.4557,523,501,217,507,437.00
4,SMACross,5,14,-0.3053,39.2423,97,29,68,0.2990,232.6117,-142.0268,2,9,1.6378,527,524,414,463,482.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
531,SMACross,20,46,-0.1041,32.2497,31,10,21,0.3226,408.9584,-239.0251,2,8,1.7109,413,358,282,437,372.50
532,SMACross,20,47,-0.0648,29.8744,32,8,24,0.2500,514.4823,-193.6284,2,7,2.6571,374,296,516,47,308.25
533,SMACross,20,48,-0.0557,30.1451,29,11,18,0.3793,369.3516,-250.1189,2,5,1.4767,364,305,76,499,311.00
534,SMACross,20,49,-0.0447,28.5999,29,10,19,0.3448,397.4869,-226.4542,2,5,1.7553,349,266,180,424,304.75


In [13]:
#The Winner!:
result_df[result_df["Score"]==result_df["Score"].min()]

,Name,sma_pfast,sma_pslow,Return,MaxDrawDown,TotalTrades#,WinTrades#,LossTrades#,WinRatio,AverageWin$,AverageLoss$,LongestWinStreak,LongestLossStreak,AverageWinLossRatio,RankReturn,RankMaxDrawDown,RankWinRatio,RankAverageWinLossRatio,Score
272,SMACross,12,23,0.2846,15.0147,48,19,29,0.3958,354.7075,-134.2447,3,5,2.6422,9,5,39,54,26.75


In [14]:
result_df.to_csv("./BTC_USDT_1h_SMACross.csv")